# This notebook is for preparing a dataset for SILVR.
It would be good to split a protein-ligand complex (pdb format) and convert the ligand file to a pdb format, before using OpenBabel or rdkit for removing bonding information

In [1]:
import os
# make sure to set the OE_LICENSE environment variable, the full path should be included, or else openeye will kill your kernel!
os.environ['OE_LICENSE'] = '/home/ian/oe_license.txt'
os.chmod('/home/ian/oe_license.txt', 0o755)

In [2]:
import os
from Bio.PDB import PDBList, PDBParser, Select, PDBIO
from subprocess import Popen, PIPE
import re
import logging 
from pathlib import Path
import contextlib
import subprocess
from rdkit import Chem
from rdkit.Chem import rdMolTransforms as rdmt
from rdkit.Chem import FragmentOnBonds
from rdkit.Chem.rdmolfiles import MolToXYZFile
from rdkit.Chem.rdmolfiles import MolToXYZBlock
import MDAnalysis as mda
import numpy as np
import pandas as pd
from tqdm import tqdm 
import glob
from asapdiscovery.data.backend.openeye import (
    oechem,
    oedocking,
    oegrid,
    oespruce,
    openeye_perceive_residues,
)
from asapdiscovery.modeling.schema import MoleculeComponent, MoleculeFilter
from asapdiscovery.modeling.modeling import split_openeye_mol, make_design_unit
import matplotlib.pyplot as plt
import seaborn as sns
# Set up the working directory
cwd = os.getcwd()

Using OpenEye Toolkits to extract the ligand (doesn't work very well right now)

In [3]:
# from https://github.com/choderalab/perses/blob/main/examples/moonshot-mainseries/00-prep-receptor.py#L19-L35
def read_pdb_file(pdb_file):
    print(f'Reading receptor from {pdb_file}...')

    from openeye import oechem
    ifs = oechem.oemolistream()
    #ifs.SetFlavor(oechem.OEFormat_PDB, oechem.OEIFlavor_PDB_Default | oechem.OEIFlavor_PDB_DATA | oechem.OEIFlavor_PDB_ALTLOC)  # Causes extra protons on VAL73 for x1425
    ifs.SetFlavor(oechem.OEFormat_PDB, oechem.OEIFlavor_PDB_Default | oechem.OEIFlavor_PDB_DATA )

    if not ifs.open(pdb_file):
        oechem.OEThrow.Fatal("Unable to open %s for reading." % pdb_file)

    mol = oechem.OEGraphMol()
    if not oechem.OEReadMolecule(ifs, mol):
        oechem.OEThrow.Fatal("Unable to read molecule from %s." % pdb_file)
    ifs.close()

    return (mol)

# script from extract_ligand_oedu.py: https://docs.eyesopen.com/toolkits/python/oechemtk/oebio_examples/oebio_example_extract_ligand.html#section-example-oebio-extract-ligand
def ExtractLigandFromDU(du, ofs):
# @ <SNIPPET-ExtractLigandFromDesignUnit>
    lig = oechem.OEGraphMol()
    if not du.GetLigand(lig):
        # TODO: this is likely to fail, how to handle this?
        oechem.OEThrow.Fatal("Error: Could not extract ligand from the OEDesignUnit.")
        
    oechem.OEWriteMolecule(ofs,lig) 
# @ </SNIPPET-ExtractLigandFromDesignUnit>

    ofs.close()

Define some classes and functions to download, rename and split the ligand and the receptor from a complex.<br>
For ligand, we are using OpenEye to extract, for receptors (proteins), we use a customisable biopython module. (Please do not use biopython to extract ligands, as it does not retain the position of the ligand atoms)

In [4]:
# using Biopython to extract the receptor
class ReceptorSelect(Select):
    '''This class is used to select only the receptor residues from a PDB file, by excluding any HETATM and any water.'''
    # I deleted the heavy metal ions as they are not a valid AutoDock type!
    # def __init__(self):
    #     # List of common heavy metal ions in protein structures
    #     self.heavy_metal_ions = ['ZN', 'FE', 'MG', 'CA', 'MN', 'CO', 'CU', 'NI', 'MO', 'W','YB']

    def accept_residue(self, residue):
        # Exclude water molecules, assumed to have residue name 'HOH' or 'WAT'
        if residue.get_resname() in ['HOH', 'WAT']:
            return False
        
        # Include heavy metal ions
        # if residue.get_resname() in self.heavy_metal_ions:
        #     return True
        
        # Check if any atom in the residue is from an ATOM record
        for atom in residue:
            '''in biopython, atom.get_full_id()[3] is accessing the fourth element of the full ID. This element represents the atom name, atom.get_full_id()[3][0] is accessing the first character of the atom name. This character represents the element symbol of the atom. The condition atom.get_full_id()[3][0] == ' ' checks whether the first character of the atom name is a space. If it is a space, then the atom is from an ATOM record, otherwise it is from a HETATM record.'''
            if atom.get_full_id()[3][0] == ' ':
                return True
        return False

class ProteinLigandSplitter:
    ''' This class is for converting a protein-ligand complex (pdb format) to a receptor and a ligand (pdb format). The receptor is saved as a separate file, and the ligand is saved as a separate file. The class uses Biopython to extract the receptor and OpenEye to extract the ligand.'''
    
    def __init__(self, input_data, is_pdb_id=True):
        self.input_data = input_data
        self.is_pdb_id = is_pdb_id
        self.pdbl = PDBList()
        self.parser = PDBParser()
        self.filename = None
        self.structure = None
        self.receptor_file = f"{self.input_data}_receptor.pdb"
        self.ligand_file = f"{self.input_data}_ligand.pdb"
        
        if self.is_pdb_id:
            self.download_pdb_file(self.input_data)
        else:
            self.filename = self.input_data
            self.structure = self.parser.get_structure('structure', self.filename)

    def download_pdb_file(self, pdb_id):
        self.filename = self.pdbl.retrieve_pdb_file(pdb_id, file_format='pdb', pdir='.', overwrite=True)
        self.structure = self.parser.get_structure('structure', self.filename)
    
    def extract_receptor(self):
        io = PDBIO()
        io.set_structure(self.structure)
        io.save(self.receptor_file, ReceptorSelect())

    def rename_downloaded_file(self):
        new_filename = f"{self.input_data}.pdb"
        if os.path.exists(self.filename) and not os.path.exists(new_filename):
            os.rename(self.filename, new_filename)
        self.filename = new_filename

    def extract_ligand(self):
        # debug line
        print(f'Extracting ligand from {self.filename}...')
        complex = read_pdb_file(self.filename)
        # du: OEDesignUnit
        # TODO: error trap for 'success'
        try:
            success, du = make_design_unit(complex)
            # use script from extract_ligand_oedu.py to extract ligand from complex
            oeoutfile = oechem.oemolostream(self.ligand_file)
            ExtractLigandFromDU(du, oeoutfile)
        except Exception as e:
            subprocess.run(f'../../docking/extract_ligand.sh {self.input_data}', shell=True)
            print(f'Error processing {self.input_data}: {e}, using my own shell script to extract ligand instead...')
            
    def split_protein_ligand(self):
        if self.is_pdb_id:
            self.rename_downloaded_file()
        self.extract_receptor()
        self.extract_ligand()
        print(f'The receptor file has been saved as {self.receptor_file}')
        print(f'The ligand file has been saved as {self.ligand_file}')
        return self.receptor_file, self.ligand_file

Use my own shell script to extract ligand (working pipeline, but more robustness testing required.)
Use OpenBabel to fragment the ligands
***Only run this once!***

In [19]:
# (try to) extract ligands from all complexes
complex_ids = glob.glob('./Mpro_complexes/*.pdb')
complex_ids = [complex_id.rsplit('.', 1)[0] for complex_id in complex_ids]
for complex_id in complex_ids:
    print(f'Processing {complex_id}...')
    try:
        # the OpenEye Toolkit isn't working very well to extract ligands for our complexes
        subprocess.run(f'../../docking/extract_ligand.sh {complex_id}', shell=True)
        print(f'Ligand extracted successfully and saved as {complex_id}_ligand.pdb.')

        # use obabel to convert to xyz format
        subprocess.run(f'obabel {complex_id}_ligand.pdb -O {complex_id}_fragment.xyz', shell=True)
        print(f'Converted ligand to xyz format and saved as {complex_id}_fragment.xyz.')
    except Exception as e:
        print(f'Error processing {complex_id}: {e}')
        continue

Processing ./Mpro_complexes/Mpro-x0305_0...
Ligand extraction completed. Output saved to ./Mpro_complexes/Mpro-x0305_0_ligand.pdb
Ligand extracted successfully and saved as ./Mpro_complexes/Mpro-x0305_0_ligand.pdb.
Converted ligand to xyz format and saved as ./Mpro_complexes/Mpro-x0305_0_fragment.xyz.
Processing ./Mpro_complexes/Mpro-x2193_0...
Ligand extraction completed. Output saved to ./Mpro_complexes/Mpro-x2193_0_ligand.pdb
Ligand extracted successfully and saved as ./Mpro_complexes/Mpro-x2193_0_ligand.pdb.
Converted ligand to xyz format and saved as ./Mpro_complexes/Mpro-x2193_0_fragment.xyz.
Processing ./Mpro_complexes/Mpro-x0397_0...
Ligand extraction completed. Output saved to ./Mpro_complexes/Mpro-x0397_0_ligand.pdb
Ligand extracted successfully and saved as ./Mpro_complexes/Mpro-x0397_0_ligand.pdb.
Converted ligand to xyz format and saved as ./Mpro_complexes/Mpro-x0397_0_fragment.xyz.
Processing ./Mpro_complexes/Mpro-x0946_0...
Ligand extraction completed. Output saved to ./

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted


Ligand extracted successfully and saved as ./Mpro_complexes/Mpro-x1077_0_ligand.pdb.
Converted ligand to xyz format and saved as ./Mpro_complexes/Mpro-x1077_0_fragment.xyz.
Processing ./Mpro_complexes/Mpro-x0395_0...
Ligand extraction completed. Output saved to ./Mpro_complexes/Mpro-x0395_0_ligand.pdb
Ligand extracted successfully and saved as ./Mpro_complexes/Mpro-x0395_0_ligand.pdb.
Converted ligand to xyz format and saved as ./Mpro_complexes/Mpro-x0395_0_fragment.xyz.
Processing ./Mpro_complexes/Mpro-x0540_0...
Ligand extraction completed. Output saved to ./Mpro_complexes/Mpro-x0540_0_ligand.pdb
Ligand extracted successfully and saved as ./Mpro_complexes/Mpro-x0540_0_ligand.pdb.
Converted ligand to xyz format and saved as ./Mpro_complexes/Mpro-x0540_0_fragment.xyz.
Processing ./Mpro_complexes/Mpro-x0995_0...
Ligand extraction completed. Output saved to ./Mpro_complexes/Mpro-x0995_0_ligand.pdb
Ligand extracted successfully and saved as ./Mpro_complexes/Mpro-x0995_0_ligand.pdb.
Conver

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted


Ligand extraction completed. Output saved to ./Mpro_complexes/Mpro-x1093_0_ligand.pdb
Ligand extracted successfully and saved as ./Mpro_complexes/Mpro-x1093_0_ligand.pdb.
Converted ligand to xyz format and saved as ./Mpro_complexes/Mpro-x1093_0_fragment.xyz.
Processing ./Mpro_complexes/Mpro-x0072_0...
Ligand extraction completed. Output saved to ./Mpro_complexes/Mpro-x0072_0_ligand.pdb
Ligand extracted successfully and saved as ./Mpro_complexes/Mpro-x0072_0_ligand.pdb.
Converted ligand to xyz format and saved as ./Mpro_complexes/Mpro-x0072_0_fragment.xyz.
Processing ./Mpro_complexes/Mpro-x0426_0...
Ligand extraction completed. Output saved to ./Mpro_complexes/Mpro-x0426_0_ligand.pdb
Ligand extracted successfully and saved as ./Mpro_complexes/Mpro-x0426_0_ligand.pdb.
Converted ligand to xyz format and saved as ./Mpro_complexes/Mpro-x0426_0_fragment.xyz.
Processing ./Mpro_complexes/Mpro-x0387_0...
Ligand extraction completed. Output saved to ./Mpro_complexes/Mpro-x0387_0_ligand.pdb
Ligan

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is ./Mpro_complexes/Mpro-x0104_0_ligand.pdb)

1 molecule converted
1 molecule converted


Converted ligand to xyz format and saved as ./Mpro_complexes/Mpro-x0104_0_fragment.xyz.
Processing ./Mpro_complexes/Mpro-x0874_0...
Ligand extraction completed. Output saved to ./Mpro_complexes/Mpro-x0874_0_ligand.pdb
Ligand extracted successfully and saved as ./Mpro_complexes/Mpro-x0874_0_ligand.pdb.
Converted ligand to xyz format and saved as ./Mpro_complexes/Mpro-x0874_0_fragment.xyz.
Processing ./Mpro_complexes/Mpro-x0967_0...
Ligand extraction completed. Output saved to ./Mpro_complexes/Mpro-x0967_0_ligand.pdb
Ligand extracted successfully and saved as ./Mpro_complexes/Mpro-x0967_0_ligand.pdb.
Converted ligand to xyz format and saved as ./Mpro_complexes/Mpro-x0967_0_fragment.xyz.
Processing ./Mpro_complexes/Mpro-x0354_0...
Ligand extraction completed. Output saved to ./Mpro_complexes/Mpro-x0354_0_ligand.pdb
Ligand extracted successfully and saved as ./Mpro_complexes/Mpro-x0354_0_ligand.pdb.
Converted ligand to xyz format and saved as ./Mpro_complexes/Mpro-x0354_0_fragment.xyz.


1 molecule converted
1 molecule converted


In [7]:
import glob
mpro_ligands = glob.glob('./mpro_ligands/*.pdb')
mpro_ligands

['./mpro_ligands/Mpro-x0946_0_ligand.pdb',
 './mpro_ligands/Mpro-x2193_0_ligand.pdb',
 './mpro_ligands/Mpro-x1077_0_ligand.pdb',
 './mpro_ligands/Mpro-x0161_0_ligand.pdb',
 './mpro_ligands/Mpro-x0434_0_ligand.pdb',
 './mpro_ligands/Mpro-x0397_0_ligand.pdb',
 './mpro_ligands/Mpro-x0305_0_ligand.pdb',
 './mpro_ligands/Mpro-x1249_0_ligand.pdb',
 './mpro_ligands/Mpro-x0426_0_ligand.pdb',
 './mpro_ligands/Mpro-x0354_0_ligand.pdb',
 './mpro_ligands/Mpro-x1093_0_ligand.pdb',
 './mpro_ligands/Mpro-x0387_0_ligand.pdb',
 './mpro_ligands/Mpro-x0874_0_ligand.pdb',
 './mpro_ligands/Mpro-x0991_0_ligand.pdb',
 './mpro_ligands/Mpro-x0678_0_ligand.pdb',
 './mpro_ligands/Mpro-x0540_0_ligand.pdb',
 './mpro_ligands/Mpro-x0995_0_ligand.pdb',
 './mpro_ligands/Mpro-x0104_0_ligand.pdb',
 './mpro_ligands/Mpro-x0072_0_ligand.pdb',
 './mpro_ligands/Mpro-x0195_0_ligand.pdb',
 './mpro_ligands/Mpro-x0107_0_ligand.pdb',
 './mpro_ligands/Mpro-x0395_0_ligand.pdb',
 './mpro_ligands/Mpro-x0967_0_ligand.pdb']

In [12]:
for ligand in mpro_ligands:
    ligand_name = ligand.rsplit('.', 1)[0]
    print(ligand_name)

./mpro_ligands/Mpro-x0946_0_ligand
./mpro_ligands/Mpro-x2193_0_ligand
./mpro_ligands/Mpro-x1077_0_ligand
./mpro_ligands/Mpro-x0161_0_ligand
./mpro_ligands/Mpro-x0434_0_ligand
./mpro_ligands/Mpro-x0397_0_ligand
./mpro_ligands/Mpro-x0305_0_ligand
./mpro_ligands/Mpro-x1249_0_ligand
./mpro_ligands/Mpro-x0426_0_ligand
./mpro_ligands/Mpro-x0354_0_ligand
./mpro_ligands/Mpro-x1093_0_ligand
./mpro_ligands/Mpro-x0387_0_ligand
./mpro_ligands/Mpro-x0874_0_ligand
./mpro_ligands/Mpro-x0991_0_ligand
./mpro_ligands/Mpro-x0678_0_ligand
./mpro_ligands/Mpro-x0540_0_ligand
./mpro_ligands/Mpro-x0995_0_ligand
./mpro_ligands/Mpro-x0104_0_ligand
./mpro_ligands/Mpro-x0072_0_ligand
./mpro_ligands/Mpro-x0195_0_ligand
./mpro_ligands/Mpro-x0107_0_ligand
./mpro_ligands/Mpro-x0395_0_ligand
./mpro_ligands/Mpro-x0967_0_ligand


# if you already have split your protein and ligands

In [5]:
import glob
ndm_1_ligands = glob.glob('./ndm_1_ligands/*.sdf')
for ligand in ndm_1_ligands:
    ligand_name = ligand.rsplit('.', 1)[0]
    # use obabel to convert to xyz format
    os.system(f'obabel {ligand} -O {ligand_name}_H_fragment.xyz -h')


*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file (title is obj01)

1 molecule converted
1 molecule converted
